# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
!pip install citipy

In [84]:
# Dependencies
import json
import requests
import pandas as pd
from api_keys import weather_api_key, g_key
from citipy import citipy
import numpy as np
from scipy.stats import linregress
from scipy import stats
from pprint import pprint
import os
import datetime as dt

#file
data = "../Output/cities.csv"



## Generate Cities List

In [85]:
# Ranges of coords
range_lat = (-90, 90)
range_lng = (-180, 180)

# List for holding coords and cities
coordinates = []
cities = []

#gather random coords
np.random.seed(1000)

lats = np.random.uniform(range_lat[0], range_lat[1], size=2000)
longs = lngs = np.random.uniform(range_lng[0], range_lng[1], size=2000)

coordinates = zip(lats, longs)

#Nearest city
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

# If the city is unique, then add it to cities list
    if city not in cities:
        cities.append(city)
        


In [86]:
# Print the city count to confirm sufficient count
len(cities)

759

In [90]:
# Convert city list to dataframe and rename column to "city"
weather_dict = {
    "City": cities,
}
weather_data = pd.DataFrame(weather_dict)

# Add columns to cities dataframe
weather_data["Cloudiness"] = np.nan
weather_data["Country"] = np.nan
weather_data["Date"] = np.nan
weather_data["Humidity"] = np.nan
weather_data["Lat"] = np.nan
weather_data["Lng"] = np.nan
weather_data["Max Temp"] = np.nan
weather_data["Wind Speed"] = np.nan
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bluff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,norman wells,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tabiauea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,leningradskiy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [44]:
# Save config information #reference
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# city = "London" test

# Build query URL | F-strings can be used too.
# query_url = f"{url}""
query_url = url + "appid=" + weather_api_key + "&q=" + city


# Get weather data
weather_response = requests.get(query_url)
#convert to a json
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")

The weather API responded with: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 281.33, 'feels_like': 279.84, 'temp_min': 279.57, 'temp_max': 282.91, 'pressure': 1014, 'humidity': 90}, 'visibility': 10000, 'wind': {'speed': 2.47, 'deg': 273, 'gust': 5.87}, 'clouds': {'all': 76}, 'dt': 1621984979, 'sys': {'type': 2, 'id': 2019646, 'country': 'GB', 'sunrise': 1622001276, 'sunset': 1622059251}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [45]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

# Use iterrows to loop through dataframe using the city name
for city in cities:
    # Build query URL | F-strings can be used too.
    # query_url = f"{url}""
    query_url = url + "appid=" + weather_api_key + "&q=" + city

    # Get weather data
    weather_response = requests.get(query_url)
    #convert to a json
    weather_json = weather_response.json()

    # Get the temperature from the response
    print(f"The weather API responded with: {weather_json}." + "\n")


The weather API responded with: {'coord': {'lon': 169.8464, 'lat': -46.2817}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 277.55, 'feels_like': 276.52, 'temp_min': 277.55, 'temp_max': 277.55, 'pressure': 1024, 'humidity': 76, 'sea_level': 1024, 'grnd_level': 1023}, 'visibility': 10000, 'wind': {'speed': 1.42, 'deg': 37, 'gust': 2.25}, 'clouds': {'all': 100}, 'dt': 1621985405, 'sys': {'type': 2, 'id': 2000217, 'country': 'NZ', 'sunrise': 1621973237, 'sunset': 1622005697}, 'timezone': 43200, 'id': 2208248, 'name': 'Kaitangata', 'cod': 200}.



### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [46]:
#  Get the indices of cities that have humidity over 100%.


In [47]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression